In [4]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.text_cell_render.rendered_html{font-size:14pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:30px;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

# red wine 품질 등급 예측

```
* 1. 데이터셋 확보 & 전처리
   독립변수와 타겟변수 분리 -> 독립변수 스케일 조정(StandardScaler),  
    -> 타겟변수 원핫인코딩(pd.get_dummies) -> 훈련셋과 테스트셋 분리(train_test_split)
* 2. 모델 구성(입력11, 출력?)
* 3. 모델 학습과정 설정(다중분류로 설정)
* 4. 모델 학습(callbacks 이용)
* 5. 모델 평가 - 그래프, 평가(테스트셋), 교차표
* 6. 모델 저장/사용
```

In [23]:
import numpy as np
import pandas as pd   # read_csv, get_dummies, crosstab  ## 파일 읽어오기위해
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model  # 모델관련
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras import metrics  # compile
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
import matplotlib.pyplot as plt   # 학습과정 시각화

## 1. 데이터셋 확보 & 전처리
- 독립변수와 타겟변수 분리 -> 독립변수 스케일 조정(StandardScaler),  
    -> 타겟변수 원핫인코딩(pd.get_dummies) -> 훈련셋과 테스트셋 분리(train_test_split)

In [24]:
# 데이터 읽어오기
# np.loadtxt('data/winequality-red.csv', delimiter=';', skiprows=1)
# np.genfromtxt('data/winequality-red.csv', delimiter=';', skip_header=1)
redwine = pd.read_csv('./data/winequality-red.csv', sep=';')
# 고정 산도(fixed acidity)
# 휘발성 산도(volatile acidity)
# 구연산(citric acid)
# 잔여 당분(residual sugar)
# 염화물(chlorides)
# 유리 아황산염(free sulfur dioxide)
# 총 아황산염(total sulfur dioxide)
# 밀도(density)
# pH
# 황산염(sulphates)
# 알코올(alcohol)
# -- 타겟 변수 : quality
# redwine.info()  ## 결측치 확인(없음)

In [25]:
# 타겟변수의 균형 확인
redwine['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [26]:
# 독립변수, 타겟변수 분리
X_redwine = redwine.iloc[:, :-1].values  # to_numpy와 유사. numpy배열로
y_redwine = redwine.iloc[:, -1]  # 시리즈  3,4,5,6,7,8
X_redwine.shape, y_redwine.shape

((1599, 11), (1599,))

In [27]:
# 독립변수 X의 스케일 조정
scaler = StandardScaler()   # 또는 MinmaxScaler()
scaler.fit(X_redwine)
scaled_X_redwine = scaler.transform(X_redwine)

# 다중분류를 위한 타겟변수의 원핫인코딩 -> numpy배열로 형변환
Y_redwine = pd.get_dummies(y_redwine).to_numpy()
# Y_redwine

In [28]:
# 독립변수:scaled_X_redwine  & 타겟변수:Y_redwine
scaled_X_redwine.shape, Y_redwine.shape

((1599, 11), (1599, 6))